# IPython Blockfolio

This is a simple "Blockfolio" using Python and the [CoinMarketCap API](https://coinmarketcap.com/api/).
You need an API key - which you can get for free - to use this notebook!

It will show the net worth in USD of your Cryptocurrency investments as well as the relative share
of coins you own in comparison to the total coin supply of the corresponding digital asset.

First, the current exchanges rates are loaded, then combined with your investments, and finally displayed using Plotly.

This notebook can be found on [my github profile](https://github.com/SmokinCaterpillar/blockfolio).

Imports
--------

In [1]:
from datetime import datetime
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

## Loading Data from CoinMarkteCap

In [2]:
LIMIT = 1000  # only load the top 1000 currencies, if you have invested in smaller ones increase this limit
API_KEY = ""  # add your API key here

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':LIMIT,
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': API_KEY,
}

session = Session()
session.headers.update(headers)

try:
    # load the json data from the API
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
except Exception as e:
    print(e)
if data["status"]["error_message"]:
    print(data["status"]["error_message"])

In [3]:
# transform data using pandas and keep track of the time
now = datetime.now()
market_data = pd.DataFrame(data['data'])
market_data.head(10)
market_data["price_usd"] = market_data.quote.apply(lambda x: x['USD']['price'])
market_data.head()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,cmc_rank,last_updated,quote,price_usd
0,1,Bitcoin,BTC,bitcoin,9893,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.865906e+07,1.865906e+07,None,1,2021-03-20T11:03:02.000Z,"{'USD': {'price': 58973.04022551859, 'volume_2...",58973.040226
1,1027,Ethereum,ETH,ethereum,6247,2015-08-07T00:00:00.000Z,"[mineable, pow, smart-contracts, ethereum, coi...",NaN,1.151261e+08,1.151261e+08,"{'id': 2502, 'name': 'Heco', 'symbol': 'HT', '...",2,2021-03-20T11:03:02.000Z,"{'USD': {'price': 1844.620233314952, 'volume_2...",1844.620233
2,1839,Binance Coin,BNB,binance-coin,604,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",1.705328e+08,1.545328e+08,1.705328e+08,None,3,2021-03-20T11:02:08.000Z,"{'USD': {'price': 270.76507535852016, 'volume_...",270.765075
3,2010,Cardano,ADA,cardano,260,2017-10-01T00:00:00.000Z,"[mineable, dpos, pos, platform, research, smar...",4.500000e+10,3.194831e+10,4.500000e+10,None,4,2021-03-20T11:02:10.000Z,"{'USD': {'price': 1.27507221641323, 'volume_24...",1.275072
4,825,Tether,USDT,tether,11564,2015-02-25T00:00:00.000Z,"[store-of-value, payments, stablecoin, stablec...",NaN,3.943707e+10,4.063629e+10,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",5,2021-03-20T11:02:08.000Z,"{'USD': {'price': 1.00328262545705, 'volume_24...",1.003283


## Your Blockfolio

Below you can fill in the types and amount of coins you have into the Python dictionary
with the format `'SYMBOL': value`.

In [4]:
# enter your coins below:
block_folio = {
    'BTC': 0.31337,
    'BCH': 0.01337,
    'ETH': 3,
    'LTC': 1.5,
    'MIOTA': 777,
    'XRP': 13,
    'DASH': 0.1, 
    'XMR': 1, 
    'LSK': 12, 
    'OMG': 1.1, 
    'XRB': 42, 
    'PIVX': 123, 
    'ARK': 22, 
    'NEO': 7, 
    'GAS': 0.01,
    'STEEM': 499,
}

# display your blockfolio
block_folio = pd.DataFrame(list(block_folio.items()), columns=['symbol', 'amount'])
block_folio.head(len(block_folio))

,symbol,amount
0,BTC,0.31337
1,BCH,0.01337
2,ETH,3.00000
3,LTC,1.50000
4,MIOTA,777.00000
5,XRP,13.00000
6,DASH,0.10000
7,XMR,1.00000
8,LSK,12.00000
9,OMG,1.10000


### Combining the Blockfolio and the API Data

In [5]:
# merge the API and blockfolio data and sort by investment value
merged_data = block_folio.merge(market_data, how='left')
merged_data['value_usd'] = merged_data.amount * merged_data.price_usd
merged_data['coinshare'] = merged_data.amount / merged_data.total_supply
merged_data = merged_data.sort_values('value_usd', ascending=False)
merged_data.head()

,symbol,amount,id,name,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,cmc_rank,last_updated,quote,price_usd,value_usd,coinshare
0,BTC,0.31337,1.0,Bitcoin,bitcoin,9893.0,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.865906e+07,1.865906e+07,None,1.0,2021-03-20T11:03:02.000Z,"{'USD': {'price': 58973.04022551859, 'volume_2...",58973.040226,18480.381615,1.679452e-08
2,ETH,3.00000,1027.0,Ethereum,ethereum,6247.0,2015-08-07T00:00:00.000Z,"[mineable, pow, smart-contracts, ethereum, coi...",NaN,1.151261e+08,1.151261e+08,"{'id': 2502, 'name': 'Heco', 'symbol': 'HT', '...",2.0,2021-03-20T11:03:02.000Z,"{'USD': {'price': 1844.620233314952, 'volume_2...",1844.620233,5533.860700,2.605839e-08
4,MIOTA,777.00000,1720.0,IOTA,iota,71.0,2017-06-13T00:00:00.000Z,"[dag, medium-of-exchange, iot, sharing-economy]",2.779530e+09,2.779530e+09,2.779530e+09,None,27.0,2021-03-20T11:03:07.000Z,"{'USD': {'price': 1.45918985345932, 'volume_24...",1.459190,1133.790516,2.795436e-07
13,NEO,7.00000,1376.0,Neo,neo,276.0,2016-09-08T00:00:00.000Z,"[platform, enterprise-solutions, smart-contracts]",1.000000e+08,7.053883e+07,1.000000e+08,None,38.0,2021-03-20T11:03:04.000Z,"{'USD': {'price': 45.23708045638193, 'volume_2...",45.237080,316.659563,7.000000e-08
15,STEEM,499.00000,1230.0,Steem,steem,36.0,2016-04-18T00:00:00.000Z,"[media, content-creation]",NaN,3.762131e+08,3.931872e+08,None,171.0,2021-03-20T11:03:03.000Z,"{'USD': {'price': 0.62989761467306, 'volume_24...",0.629898,314.318910,1.269116e-06


### Your Blockfolio in numbers and graphs!


In [6]:
networth = 'Your blockfolio is currently (i.e {}) worth {:.2f} USD!'.format(now.strftime('%Y-%m-%d %I:%M %p'), 
                                                                        merged_data.value_usd.sum())
print(networth)

Your blockfolio is currently (i.e 2021-03-20 12:03 PM) worth 26632.95 USD!


#### Net Worth

Let's plot the worth of each asset in descending order:

In [7]:
marker = dict(color='rgb(158,202,225)',
              line=dict(color='rgb(8,48,107)', width=1.5))

layout = go.Layout(title='Blockfolio Networth in USD',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='USD'))
              
value_chart=go.Bar(x=merged_data.symbol, y=merged_data.value_usd, marker=marker)

fig = go.Figure(data=[value_chart], layout=layout)
py.iplot(fig)

#### Fraction of total Supply
Finally, that looks at the relative sizes of your investment. 
This gives you an idea or visualizes in which cryptocurrency you believe in the most. 
The graph below shows you fractional share of the the current crypto's total supply.

In [8]:
marker = dict(color='rgb(258,202,125)', line=dict(color='rgb(8,48,107)', width=1.5,))

layout = go.Layout(title='Relative Blockfolio Size',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='Fraction of total Supply'))

share_chart=go.Bar(x=merged_data.symbol, y=merged_data.coinshare, marker=marker)

fig = go.Figure(data=[share_chart], layout=layout)
py.iplot(fig)